# Quiz 4

In [1]:
import turicreate

In [2]:
people = turicreate.SFrame('data/people_wiki.sframe')

In [3]:
# Conteo de palabras en cada texto
people['word_count'] = turicreate.text_analytics.count_words(people['text'])

In [4]:
# tf-idf es una medida numérica que expresa cuán relevante es una palabra
people['tfidf'] = turicreate.text_analytics.tf_idf(people['text'])

## Pregunta 1

1. **Compare las palabras principales según el recuento de palabras con TF-IDF:** en el cuaderno que cubrimos en el módulo, exploramos dos representaciones de documentos: recuento de palabras y TF-IDF. Ahora, tomemos a una persona famosa en particular, 'Elton John'. 
+ ¿Cuáles son las 3 palabras de sus artículos con mayor número de palabras? 
+ ¿Cuáles son las 3 palabras de sus artículos con mayor TF-IDF? 

    Estos resultados ilustran por qué TF-IDF es útil para encontrar palabras importantes.

In [5]:
# Seleccionamos a Elton John
elton = people[people['name'] == 'Elton John']

In [6]:
# Se muestran las 3 palabras mas repetidas en su texto descriptivo.
elton_table_word_count = elton[['word_count']].stack(
    'word_count', new_column_name = ['word','count']).sort(
    'count',ascending=False)
elton_table_word_count.head(3)

word,count
the,27.0
in,18.0
and,15.0


In [7]:
# Se muestran las 3 palabras mas relevantes en su texto descriptivo.
elton_table_tfidf = elton[['tfidf']].stack(
    'tfidf', new_column_name = ['word','tfidf']).sort(
    'tfidf',ascending=False)
elton_table_tfidf.head(3)

word,tfidf
furnish,18.38947183999428
elton,17.482320270031995
billboard,17.30368095754203


## Pregunta 2

**Medir la distancia:** Elton John es un cantante famoso; Calculemos la distancia entre su artículo y los de otros dos cantantes famosos. En esta tarea, usará la distancia del coseno, que es una medida de similitud entre vectores, similar a la que se discutió en las lecciones. Puede calcular esta distancia usando la función `turicreate.distances.cosine`. 
+ ¿Cuál es la distancia de coseno entre los artículos sobre "Elton John" y "Victoria Beckham"?
+ ¿Cuál es la distancia coseno entre los artículos sobre "Elton John" y "Paul McCartney "?
+ ¿Cuál de los dos está más cerca de Elton John? ¿Tiene sentido este resultado para usted? 

In [8]:
victoria = people[people['name'] == 'Victoria Beckham']
paul = people[people['name'] == 'Paul McCartney']

In [9]:
# distancia entre Elton y Victoria
turicreate.distances.cosine(elton['tfidf'][0],victoria['tfidf'][0])

0.9567006376655429

In [10]:
# distancia entre Elton y Paul
turicreate.distances.cosine(elton['tfidf'][0],paul['tfidf'][0])

0.8250310029221779

## Pregunta 3

Construir modelos de vecinos más cercanos con diferentes características de entrada y establecer la métrica de distancia: en el cuaderno de muestra, construimos un modelo de vecinos más cercanos para recuperar artículos usando TF-IDF como características y usando la configuración predeterminada en la construcción del modelo de vecinos más cercanos. Ahora, construirá dos modelos de vecinos más cercanos:
+ Usar recuentos de palabras como características
+ Usando TF-IDF como características
En ambos modelos, vamos a establecer la función de distancia a la similitud del coseno. Así es como: cuando llamas a la función `turicreate.nearest_neighbors.create` y al parámetro `distance='cosine'`

In [11]:
word_count_cosine_model = turicreate.nearest_neighbors.create(
    people,
    features=['word_count'],
    label='name',
    distance='cosine',
    verbose=False)

In [12]:
tfidf_cosine_model = turicreate.nearest_neighbors.create(
    people,
    features=['tfidf'],
    label='name',
    distance='cosine',
    verbose=False)

Ahora estamos listos para usar nuestro modelo para recuperar documentos. Utilice estos dos modelos para recopilar los siguientes resultados:

+ **¿Cuál es el artículo más similar, además de sí mismo, al de "Elton John" que usa funciones de conteo de palabras?**

In [13]:
word_count_cosine_model.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 27.67ms      |

| Done         |         | 100         | 2.05s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,2.220446049250313e-16,1
0,Cliff Richard,0.16142415258967036,2
0,Sandro Petrone,0.16822542751041114,3
0,Rod Stewart,0.16832716558706107,4
0,Malachi O'Doherty,0.177315545978884,5


+ **¿Cuál es el artículo más similar, además de sí mismo, al de "Elton John" usando funciones TF-IDF?**

In [14]:
tfidf_cosine_model.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 46.155ms     |

| Done         |         | 100         | 2.24s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,-2.220446049250313e-16,1
0,Rod Stewart,0.7172196678927374,2
0,George Michael,0.7476009989692848,3
0,Sting (musician),0.7476719544306141,4
0,Phil Collins,0.7511932487904706,5


+ **¿Cuál es el artículo más similar, además de él mismo, al de "Victoria Beckham" que usa funciones de conteo de palabras?**

In [15]:
word_count_cosine_model.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 43.91ms      |

| Done         |         | 100         | 1.63s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,-2.220446049250313e-16,1
0,Mary Fitzgerald (artist),0.20730703611504997,2
0,Adrienne Corri,0.21450978278754795,3
0,Beverly Jane Fry,0.21746646874079278,4
0,Raman Mundair,0.21769547499150488,5


+ **¿Cuál es el artículo más similar, además de él mismo, al de "Victoria Beckham" que utiliza funciones de TF-IDF?**

In [16]:
tfidf_cosine_model.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 50.589ms     |

| Done         |         | 100         | 1.98s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,1.1102230246251565e-16,1
0,David Beckham,0.5481696102632145,2
0,Stephen Dow Beckham,0.7849867068283364,3
0,Mel B,0.8095855234085036,4
0,Caroline Rush,0.81982642291868,5
